In [ ]:
# Load some required libraries
import pandas as pd
import numpy as np
import tensorflow.compat.v1 as tf
import keras
import matplotlib.pyplot as plt
import math
import time
import warnings
warnings.filterwarnings('ignore')
plt.style.use('fivethirtyeight')
from sklearn import metrics
import numpy as np
import pandas as pd
import math
import sklearn
import sklearn.preprocessing
import datetime
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping


from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, GRU, Bidirectional
from sklearn.metrics import mean_squared_error, r2_score
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf
import plotly.offline as pyo
cf.go_offline()
pyo.init_notebook_mode()
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
print(__version__)
import os
for dirname, _, filenames in os.walk('kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Loading our data

In [ ]:
df = pd.read_csv("../input/nyse/prices-split-adjusted.csv")
plot_x = df['date'].copy()
df.set_index("date", inplace = True)
df.index = pd.to_datetime(df.index)
df.head()


# Checking for duplicates and null values

In [ ]:
#Check for duplicated values
df.duplicated().sum()

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
#Check for null values
df.isna().sum()

In [ ]:
# Calculate the unique symbols
symbols = list(set(df.symbol))
len(symbols)

In [ ]:
df['close'].plot(figsize=(12, 6))
plt.title('Stock Prices')
plt.xlabel('Date')
plt.ylabel('Closing Price')
plt.show()

In [ ]:
df['volume'].hist(figsize=(12, 6))
plt.title('Volume Distribution')
plt.xlabel('Volume')
plt.ylabel('Frequency')
plt.show()

# Preprocessing

In [ ]:
plt.figure(figsize=(15, 5));
plt.subplot(1,2,1);
plt.plot(df[df.symbol == 'EQIX'].open.values, color='red', label='open')
plt.plot(df[df.symbol == 'EQIX'].close.values, color='green', label='close')
plt.plot(df[df.symbol == 'EQIX'].low.values, color='blue', label='low')
plt.plot(df[df.symbol == 'EQIX'].high.values, color='black', label='high')
plt.title('stock price')
plt.xlabel('time [days]')
plt.ylabel('price')
plt.legend(loc='best')
#plt.show()

plt.subplot(1,2,2);
plt.plot(df[df.symbol == 'EQIX'].volume.values, color='black', label='volume')
plt.title('stock volume')
plt.xlabel('time [days]')
plt.ylabel('volume')
plt.legend(loc='best');

In [ ]:
df.symbol.value_counts()

We will choose one specific stock to build our model so will choose **KSU** stock

In [ ]:
# Scaling
# Selecting the stock with symbol 'KSU'
KSU_stock = df[df['symbol'] == 'KSU']

# Initialize the MinMaxScaler for feature scaling
x_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()

# Create a copy of the KSU stock dataframe
KSU_df = KSU_stock.copy()

# Remove the 'symbol' column from the dataframe
KSU_df.drop(['symbol'], axis=1, inplace=True)

# Separate the features (x) and the target variable (y)
x = KSU_df[['open', 'low', 'high', 'volume']].copy()
y = KSU_df['close'].copy()

# Scale the features using the x_scaler
x[['open', 'low', 'high', 'volume']] = x_scaler.fit_transform(x)

# Reshape and scale the target variable using the y_scaler
y = y_scaler.fit_transform(y.values.reshape(-1, 1))

In [ ]:
# Splitting
# Function to load and split the data into train and test sets
def load_data(X, seq_len, train_size=0.9):
    amount_of_features = X.shape[1]
    X_mat = X.values
    sequence_length = seq_len + 1
    data = []
    
    # Generate sequences of length seq_len + 1
    for index in range(len(X_mat) - sequence_length):
        data.append(X_mat[index: index + sequence_length])
    
    data = np.array(data)
    
    # Split the data into train and test sets based on train_size
    train_split = int(round(train_size * data.shape[0]))
    train_data = data[:train_split, :]
    
    # Split the train data into input features (x_train) and target variable (y_train)
    x_train = train_data[:, :-1]
    y_train = train_data[:, -1][:,-1]
    
    # Split the test data into input features (x_test) and target variable (y_test)
    x_test = data[train_split:, :-1] 
    y_test = data[train_split:, -1][:,-1]

    # Reshape the input features arrays
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], amount_of_features))
    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], amount_of_features))  

    return x_train, y_train, x_test, y_test

# Define the window size
window = 22

# Add the 'close' column as the target variable to the feature matrix x
x['close'] = y

# Load and split the data using the load_data function
X_train, y_train, X_test, y_test = load_data(x, window)

# Print the shapes of the train and test sets
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)


### Visualize our KSU stock

In [ ]:
plt.figure(figsize=(15, 5));
plt.plot(KSU_stock.open.values, color='red', label='open')
plt.plot(KSU_stock.close.values, color='green', label='low')
plt.plot(KSU_stock.low.values, color='blue', label='low')
plt.plot(KSU_stock.high.values, color='black', label='high')

#plt.plot(df_stock_norm.volume.values, color='gray', label='volume')
plt.title('stock')
plt.xlabel('time [days]')
plt.ylabel('price/volume')
plt.legend(loc='best')
plt.show()

# Modeling

In [ ]:
#LSTM architecture
model = Sequential()
# First LSTM layer with Dropout regularisation
model.add(LSTM(units=50, input_shape=(window,5),return_sequences=True))
model.add(Dropout(0.2))
# Second LSTM layer
model.add(LSTM(units=50,return_sequences=True))
model.add(Dropout(0.2))
# Third LSTM layer
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
# Fourth LSTM layer
model.add(LSTM(units=50))
model.add(Dropout(0.5))
# The output layer
model.add(Dense(units=50, kernel_initializer='uniform', activation='tanh'))
model.add(Dense(units=1, kernel_initializer='uniform', activation='linear'))

earlystop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)
callbacks_list = [earlystop]

# Compiling the RNN
model.compile(optimizer='adam',loss='mean_squared_error')
# Fitting to the training set
start = time.time()
LSTM=model.fit(X_train,y_train,epochs=100,batch_size=35, validation_split=0.05, verbose=1,callbacks=callbacks_list)
print ('compilation time : ', time.time() - start)

In [ ]:
model.summary()

In [ ]:
%matplotlib inline
losses = pd.DataFrame(LSTM.history)
losses.plot()

# Prediction

In [ ]:
# Perform predictions on the training and test sets
trainPredict = model.predict(X_train)
testPredict = model.predict(X_test)

# Inverse transform the scaled predictions to obtain actual values
trainPredict = y_scaler.inverse_transform(trainPredict)
trainY = y_scaler.inverse_transform([y_train])
testPredict = y_scaler.inverse_transform(testPredict)
testY = y_scaler.inverse_transform([y_test])

In [ ]:
# Reshape the predicted and actual values for plotting
plot_predicted = testPredict.copy()
plot_predicted = plot_predicted.reshape(174, 1)
plot_actual = testY.copy()
plot_actual = plot_actual.reshape(174, 1)

# Print the shapes of the reshaped arrays for verification
print(plot_actual.shape)
print(plot_predicted.shape)

In [ ]:
# Create a figure with a specific size for the plot
plt.figure(figsize=(20, 7))
# Convert the plot_x variable to datetime format
plot_x = pd.to_datetime(plot_x.iloc[-174:])
# Plot the predicted values
plt.plot(pd.DataFrame(plot_predicted), label='Predicted')
# Plot the actual values
plt.plot(pd.DataFrame(plot_actual), label='Actual')
# Add a legend to the plot
plt.legend(loc='best')
# Display the plot
plt.show()

In [ ]:
trainScore = metrics.mean_squared_error(trainY[0], trainPredict[:,0]) ** .5
print('Train Score: %.2f RMSE' % (trainScore))
testScore = metrics.mean_squared_error(testY[0], testPredict[:,0]) ** .5
print('Test Score: %.2f RMSE' % (testScore))

In [ ]:
KSU_stock_prices = KSU_stock.close.values.astype('float32')
KSU_stock_prices = KSU_stock_prices.reshape(len(KSU_stock_prices), 1)

In [ ]:
# Create an empty array of the same shape as KSU_stock_prices for the train predictions plot
trainPredictPlot = np.empty_like(KSU_stock_prices)
trainPredictPlot[:, :] = np.nan
# Set the predicted values in the trainPredictPlot array
trainPredictPlot[22:len(trainPredict)+22, :] = trainPredict
# Create an empty array of the same shape as KSU_stock_prices for the test predictions plot
testPredictPlot = np.empty_like(KSU_stock_prices)
testPredictPlot[:, :] = np.nan
# Set the predicted values in the testPredictPlot array
testPredictPlot[(len(KSU_stock_prices) - testPredict.shape[0]):len(KSU_stock_prices), :] = testPredict

In [ ]:
plt.figure(figsize=(20,7))
plt.plot(pd.DataFrame(KSU_stock_prices, columns=["close"], index=KSU_df.index).close, label='Actual')
plt.plot(pd.DataFrame(trainPredictPlot, columns=["close"], index=KSU_df.index).close, label='Training')
plt.plot(pd.DataFrame(testPredictPlot, columns=["close"], index=KSU_df.index).close, label='Testing')
plt.legend(loc='best')
plt.show()

In [ ]:
model.save('./Final_LSTM_model.h5')